In [25]:
import pandas as pd
import pickle

import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

In [ ]:
csv_file_path = 'C:/Users/96446/Documents/GitHub/learning Pytorch/pytorch/FB15K_Updated.csv'

df = pd.read_csv(csv_file_path)

entity_type_counts = df['Entity Type'].value_counts()
filtered_entity_type_counts = entity_type_counts[entity_type_counts >= 2]

print(filtered_entity_type_counts)

In [12]:
pkl_file_path  = 'C:/Users\96446\Documents\GitHub\data\mmkg\pkls/FBDB15K_id_img_feature_dict.pkl'
# 用pickle加载数据
with open(pkl_file_path, 'rb') as file:
    data = pickle.load(file)

# 检查数据的类型
print(type(data))

<class 'dict'>


In [24]:
data

{6047: array([0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.5510511], dtype=float32),
 14526: array([4.00271  , 3.6201923, 0.6277808, ..., 3.0336063, 0.3198567,
        0.       ], dtype=float32),
 3428: array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 13847: array([0.70729464, 3.811     , 2.2170372 , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 11953: array([0.26017112, 0.25488573, 0.        , ..., 0.        , 0.8810961 ,
        1.2724358 ], dtype=float32),
 2566: array([0.       , 3.4103918, 0.       , ..., 0.       , 0.       ,
        2.0073457], dtype=float32),
 12032: array([0.       , 0.       , 0.       , ..., 2.321962 , 0.       ,
        4.6906624], dtype=float32),
 1514: array([0.20058924, 0.        , 0.        , ..., 0.        , 0.        ,
        1.4033422 ], dtype=float32),
 12301: array([1.356245 , 5.187034 , 0.       , ..., 0.       , 0.       ,
        3.8238647], dtype=float32),
 11553: array([0.       , 3.101435 , 

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

# Define the device to run the model on (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True).to(device)

# We will use the output of the last convolutional layer
model = torch.nn.Sequential(*(list(model.children())[:-2])).to(device)

# Set the model to evaluation mode
model.eval()

# Define the image transformations required by VGG16
transform = transforms.Compose([
    transforms.Resize(256),         # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),     # Crop the image to 224x224 pixels about the center
    transforms.ToTensor(),          # Convert the image to a PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

def image_to_embedding(image_path):
    # Load and transform the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    
    # Get the embedding from the model
    with torch.no_grad():
        embedding = model(image)
        
    # Convert the embedding into a 1D tensor
    embedding = embedding.view(embedding.size(0), -1)
    return embedding.cpu().numpy()

# Example usage:
# Replace 'image_path.jpg' with the path to an image file
embedding = image_to_embedding('image_path.jpg')
print(embedding)
